In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['HA']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('hemagglutinin', 2693),
 ('hyaluronic acid', 2676),
 ('hyaluronan', 1989),
 ('hydroxyapatite', 970),
 ('humic acid', 422),
 ('haemagglutinin', 325),
 ('histamine', 275),
 ('hemagglutination', 192),
 ('hyaluronic', 147),
 ('harm avoidance', 65),
 ('hydroxylamine', 58),
 ('heat acclimation', 55),
 ('high altitude', 53),
 ('hemophilia a', 47),
 ('hemiarthroplasty', 35),
 ('hemaglutinin', 31),
 ('herbimycin a', 30),
 ('haemagglutination', 27),
 ('hypocrellin a', 25),
 ('hearing aid', 22),
 ('high affinity', 22),
 ('hepatic artery', 20),
 ('head activity', 20),
 ('ha', 18),
 ('hyperandrogenemia', 17),
 ('hemagglutinin protein', 17),
 ('hyperammonemia', 15),
 ('hyperactivation', 14),
 ('human albumin', 13),
 ('haemophilia a', 13),
 ('homotypic aggregation', 12),
 ('hemagglutination activity', 12),
 ('hyperandrogenism', 12),
 ('hydroxylapatite', 12),
 ('hemoadsorption', 11),
 ('hemagglutination assay', 10),
 ('high analgesia', 9),
 ('human astrocytes', 9),
 ('heme arginate', 9),
 ('headache

In [7]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [8]:
names

{'MESH:D006388': 'Hemagglutinins',
 'MESH:D006820': 'Hyaluronic Acid',
 'CHEBI:CHEBI:132153': 'hyaluronate',
 'MESH:D017886': 'Durapatite',
 'CHEBI:CHEBI:37527': 'acid',
 'CHEBI:CHEBI:18295': 'histamine',
 'MESH:D006384': 'Hemagglutination',
 'CHEBI:CHEBI:15429': 'hydroxylamine',
 'GO:GO:0010286': 'heat acclimation',
 'MESH:D000531': 'Altitude',
 'MESH:D006467': 'Hemophilia A',
 'MESH:D062785': 'Hemiarthroplasty',
 'CHEBI:CHEBI:5674': 'herbimycin',
 'MESH:C048388': 'hypocrellin A',
 'MESH:D006310': 'Hearing Aids',
 'MESH:D006499': 'Hepatic Artery',
 'CHEBI:CHEBI:33349': 'dubnium atom',
 'MESH:D011085': 'Polycystic Ovary Syndrome',
 'CHEBI:CHEBI:36080': 'protein',
 'MESH:D022124': 'Hyperammonemia',
 'MESH:D000075462': 'Serum Albumin, Human',
 'CHEBI:CHEBI:13193': 'hydrogen acceptor',
 'MESH:D017588': 'Hyperandrogenism',
 'CHEBI:CHEBI:52255': 'hydroxylapatite',
 'MESH:D000698': 'Analgesia',
 'MESH:C048849': 'heme arginate',
 'MESH:D006261': 'Headache',
 'CHEBI:CHEBI:5831': 'Hypaconitine'

In [9]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-31 20:08:38] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [10]:
result = [grounding_map, names, pos_labels]

In [11]:
result

[{'ha': 'ungrounded',
  'ha antibodies': 'ungrounded',
  'ha control cells': 'ungrounded',
  'ha tagged': 'ungrounded',
  'ha vector': 'ungrounded',
  'ha14 1': 'ungrounded',
  'haemagglutination': 'MESH:D006388',
  'haemagglutination activity': 'MESH:D006388',
  'haemagglutination antibodies': 'ungrounded',
  'haemagglutinin': 'MESH:D006388',
  'haemaglutinin': 'MESH:D006388',
  'haemophilia a': 'MESH:D006467',
  'haloperidol': 'CHEBI:CHEBI:5613',
  'harm avoidance': 'ungrounded',
  'harmonic average': 'ungrounded',
  'harzianum a': 'MESH:C087546',
  'head activity': 'ungrounded',
  'headache': 'MESH:D006261',
  'healthy americans': 'ungrounded',
  'hearing aid': 'MESH:D006310',
  'heat acclimation': 'ungrounded',
  'hemagglutination': 'MESH:D006388',
  'hemagglutination activity': 'ungrounded',
  'hemagglutination assay': 'ungrounded',
  'hemagglutinin': 'MESH:D006388',
  'hemagglutinin a': 'MESH:D006388',
  'hemagglutinin antigenic': 'MESH:D006388',
  'hemagglutinin epitope': 'MESH:

In [13]:
grounding_map, names, pos_labels = [{'ha': 'ungrounded',
  'ha antibodies': 'ungrounded',
  'ha control cells': 'ungrounded',
  'ha tagged': 'ungrounded',
  'ha vector': 'ungrounded',
  'ha14 1': 'ungrounded',
  'haemagglutination': 'MESH:D006388',
  'haemagglutination activity': 'MESH:D006388',
  'haemagglutination antibodies': 'ungrounded',
  'haemagglutinin': 'MESH:D006388',
  'haemaglutinin': 'MESH:D006388',
  'haemophilia a': 'MESH:D006467',
  'haloperidol': 'CHEBI:CHEBI:5613',
  'harm avoidance': 'ungrounded',
  'harmonic average': 'ungrounded',
  'harzianum a': 'MESH:C087546',
  'head activity': 'ungrounded',
  'headache': 'MESH:D006261',
  'healthy americans': 'ungrounded',
  'hearing aid': 'MESH:D006310',
  'heat acclimation': 'ungrounded',
  'hemagglutination': 'MESH:D006388',
  'hemagglutination activity': 'ungrounded',
  'hemagglutination assay': 'ungrounded',
  'hemagglutinin': 'MESH:D006388',
  'hemagglutinin a': 'MESH:D006388',
  'hemagglutinin antigenic': 'MESH:D006388',
  'hemagglutinin epitope': 'MESH:D006388',
  'hemagglutinin gene': 'MESH:D006388',
  'hemagglutinin peptide': 'MESH:D006388',
  'hemagglutinin protein': 'MESH:D006388',
  'hemaglutinin': 'MESH:D006388',
  'hemagluttinin': 'MESH:D006388',
  'hemangiogenesis': 'ungrounded',
  'hemangioma': 'MESH:D006391',
  'heme arginate': 'MESH:C048849',
  'hemi arthroplasty': 'MESH:D062785',
  'hemiarthroplasty': 'ungrounded',
  'hemoadsorption': 'ungrounded',
  'hemolytic anemia': 'DOID:DOID:583',
  'hemophilia a': 'MESH:D006467',
  'hepatic a': 'ungrounded',
  'hepatic abscess': 'MESH:D008100',
  'hepatic adenoma': 'MESH:D018248',
  'hepatic artery': 'MESH:D006499',
  'hepatocellular adenoma': 'DOID:DOID:0050868',
  'herbimycin a': 'CHEBI:CHEBI:5674',
  'heterophilic antibodies': 'ungrounded',
  'hibiscus acetosella': 'ungrounded',
  'high affinity': 'ungrounded',
  'high altitude': 'ungrounded',
  'high amylose': 'ungrounded',
  'high analgesia': 'MESH:D000698',
  'hippocampal atrophy': 'HP:HP:0410170',
  'hippocampal avoidance': 'ungrounded',
  'hispanic americans': 'ungrounded',
  'histamine': 'MESH:D006632',
  'histaminergic': 'MESH:D006632',
  'holothurin a': 'MESH:C034723',
  'home agents': 'ungrounded',
  'homotypic aggregation': 'ungrounded',
  'horizontal activity': 'ungrounded',
  'hospital acquired': 'ungrounded',
  'hospital authors': 'ungrounded',
  'human albumin': 'MESH:D000075462',
  'human amylin': 'CHEBI:CHEBI:80342',
  'human astrocytes': 'MESH:D001253',
  'human serum albumin': 'MESH:D000075462',
  'humic acid': 'MESH:D006812',
  'humoral antibodies': 'ungrounded',
  'huperzine a': 'CHEBI:CHEBI:78330',
  'hyaluronan': 'CHEBI:CHEBI:16336',
  'hyaluronic': 'CHEBI:CHEBI:16336',
  'hyaluronic acid': 'CHEBI:CHEBI:16336',
  'hyaluronic acid sodium salt': 'CHEBI:CHEBI:16336',
  'hyaluronidase': 'MESH:D006821',
  'hybrid algorithm': 'ungrounded',
  'hybrid augmentation': 'ungrounded',
  'hydroalcoholic': 'ungrounded',
  'hydroxy apatite': 'CHEBI:CHEBI:52255',
  'hydroxyapatite': 'CHEBI:CHEBI:52255',
  'hydroxyapatite ceramic': 'CHEBI:CHEBI:52255',
  'hydroxyapatite nanoparticles': 'CHEBI:CHEBI:52255',
  'hydroxyapatite particles': 'CHEBI:CHEBI:52255',
  'hydroxylamine': 'CHEBI:CHEBI:15429',
  'hydroxylapatite': 'CHEBI:CHEBI:52255',
  'hyluronan': 'CHEBI:CHEBI:16336',
  'hypaconitine': 'CHEBI:CHEBI:5831',
  'hyperactivation': 'ungrounded',
  'hyperammonemia': 'MESH:D022124',
  'hyperammonemic': 'ungrounded',
  'hyperandrogenemia': 'MESH:D017588',
  'hyperandrogenism': 'MESH:D017588',
  'hypercapnic acidosis': 'ungrounded',
  'hypocrellin a': 'MESH:C048388',
  'hypodermin a': 'CHEBI:CHEBI:13193',
  'hypothalamic amenorrhea': 'MESH:D000568',
  'hypothalamic amenorrhoea': 'ungrounded',
  'hypoxia acidosis': 'ungrounded',
  'hysteresis area': 'ungrounded'},
 {'MESH:D006388': 'Hemagglutinins',
  'MESH:D006467': 'Hemophilia A',
  'CHEBI:CHEBI:5613': 'haloperidol',
  'MESH:C087546': 'harzianum A',
  'MESH:D006261': 'Headache',
  'MESH:D006310': 'Hearing Aids',
  'MESH:D006391': 'Hemangioma',
  'MESH:C048849': 'heme arginate',
  'MESH:D062785': 'Hemiarthroplasty',
  'DOID:DOID:583': 'hemolytic anemia',
  'MESH:D008100': 'Liver Abscess',
  'MESH:D018248': 'Adenoma, Liver Cell',
  'MESH:D006499': 'Hepatic Artery',
  'DOID:DOID:0050868': 'hepatocellular adenoma',
  'CHEBI:CHEBI:5674': 'herbimycin',
  'MESH:D000698': 'Analgesia',
  'HP:HP:0410170': 'Hippocampal atrophy',
  'MESH:D006632': 'Histamine',
  'MESH:C034723': 'holothurin A',
  'MESH:D000075462': 'Serum Albumin, Human',
  'CHEBI:CHEBI:80342': 'Amylin',
  'MESH:D001253': 'Astrocytes',
  'MESH:D006812': 'Humic Substances',
  'CHEBI:CHEBI:78330': 'huperzine A',
  'CHEBI:CHEBI:16336': 'hyaluronic acid',
  'MESH:D006821': 'Hyaluronoglucosaminidase',
  'CHEBI:CHEBI:52255': 'hydroxylapatite',
  'CHEBI:CHEBI:15429': 'hydroxylamine',
  'CHEBI:CHEBI:5831': 'Hypaconitine',
  'MESH:D022124': 'Hyperammonemia',
  'MESH:D017588': 'Hyperandrogenism',
  'MESH:C048388': 'hypocrellin A',
  'CHEBI:CHEBI:13193': 'hydrogen acceptor',
  'MESH:D000568': 'Amenorrhea'},
 ['CHEBI:CHEBI:15429',
  'CHEBI:CHEBI:16336',
  'CHEBI:CHEBI:52255',
  'MESH:D006388',
  'MESH:D006632',
  'MESH:D006812']]

In [14]:
excluded_longforms = ['ha', 'ha tagged', 'ha vector', 'ha control cells']

In [15]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [19]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [20]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [21]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-31 20:29:38] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-31 20:36:18] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9467159597771889 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [22]:
classifier.stats

{'label_distribution': {'MESH:D006499': 16,
  'MESH:D006388': 2405,
  'MESH:D006632': 220,
  'CHEBI:CHEBI:15429': 52,
  'CHEBI:CHEBI:52255': 723,
  'CHEBI:CHEBI:16336': 3492,
  'MESH:C048388': 21,
  'ungrounded': 268,
  'MESH:D006812': 337,
  'DOID:DOID:0050868': 3,
  'MESH:D000075462': 15,
  'MESH:D000568': 6,
  'CHEBI:CHEBI:5674': 26,
  'CHEBI:CHEBI:13193': 6,
  'MESH:D022124': 12,
  'HP:HP:0410170': 7,
  'CHEBI:CHEBI:5613': 4,
  'CHEBI:CHEBI:78330': 4,
  'MESH:D000698': 2,
  'CHEBI:CHEBI:80342': 3,
  'MESH:D006261': 8,
  'MESH:D017588': 16,
  'MESH:C048849': 7,
  'DOID:DOID:583': 5,
  'MESH:D006467': 43,
  'MESH:D006391': 8,
  'CHEBI:CHEBI:5831': 8,
  'MESH:C034723': 3,
  'MESH:D018248': 3,
  'MESH:D001253': 7,
  'MESH:D006821': 4,
  'MESH:D006310': 16,
  'MESH:C087546': 3,
  'MESH:D008100': 1,
  'MESH:D062785': 2},
 'f1': {'mean': 0.946716, 'std': 0.004261},
 'precision': {'mean': 0.938217, 'std': 0.00427},
 'recall': {'mean': 0.956979, 'std': 0.00494},
 'CHEBI:CHEBI:13193': {'f1':

In [23]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [24]:
disamb.dump(model_name, results_path)

In [25]:
print(disamb.info())

Disambiguation model for HA

Produces the disambiguations:
	Adenoma, Liver Cell	MESH:D018248
	Amenorrhea	MESH:D000568
	Amylin	CHEBI:CHEBI:80342
	Analgesia	MESH:D000698
	Astrocytes	MESH:D001253
	Headache	MESH:D006261
	Hearing Aids	MESH:D006310
	Hemagglutinins*	MESH:D006388
	Hemangioma	MESH:D006391
	Hemiarthroplasty	MESH:D062785
	Hemophilia A	MESH:D006467
	Hepatic Artery	MESH:D006499
	Hippocampal atrophy	HP:HP:0410170
	Histamine*	MESH:D006632
	Humic Substances*	MESH:D006812
	Hyaluronoglucosaminidase	MESH:D006821
	Hypaconitine	CHEBI:CHEBI:5831
	Hyperammonemia	MESH:D022124
	Hyperandrogenism	MESH:D017588
	Liver Abscess	MESH:D008100
	Serum Albumin, Human	MESH:D000075462
	haloperidol	CHEBI:CHEBI:5613
	harzianum A	MESH:C087546
	heme arginate	MESH:C048849
	hemolytic anemia	DOID:DOID:583
	hepatocellular adenoma	DOID:DOID:0050868
	herbimycin	CHEBI:CHEBI:5674
	holothurin A	MESH:C034723
	huperzine A	CHEBI:CHEBI:78330
	hyaluronic acid*	CHEBI:CHEBI:16336
	hydrogen acceptor	CHEBI:CHEBI:13193
	hydroxyl

In [26]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1